# RecSys for VK contest

Решение Никиты Венедиктова

Необходимо построить векторные представления (эмбеддинги пользователей и фильмов), чтобы рекомендовать фильмы пользователю

1. Разбиение данных на обучение и валидацию
2. Выбор и обоснование метрики
3. Разработка архитектуры сетки
4. Обучение и реализация
5. Воспроизводимость

In [1]:
%pip install pandas matplotlib wordcloud scikit-learn

     ---------------------------------------- 10.7/10.7 MB 4.7 MB/s eta 0:00:00
     ---------------------------------------- 7.6/7.6 MB 7.3 MB/s eta 0:00:00
     -------------------------------------- 152.1/152.1 kB 8.9 MB/s eta 0:00:00
     ---------------------------------------- 8.3/8.3 MB 10.4 MB/s eta 0:00:00
     ------------------------------------- 341.8/341.8 kB 10.4 MB/s eta 0:00:00
     ------------------------------------- 502.3/502.3 kB 10.5 MB/s eta 0:00:00
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ---------------------------------------- 55.3/55.3 kB ? eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 7.1 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 9.4 MB/s eta 0:00:00
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
     -------------------------------------- 163.0/163.0 kB 9.5 MB/s eta 0:00:00
     --------------------------------------- 42.5/42.5 MB 10.7 MB/s eta 0:00:00
  Using cached joblib-1

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [11]:
movies=pd.read_csv('./ml-latest-small/movies.csv')
ratings=pd.read_csv('./ml-latest-small/ratings.csv')

In [17]:
print(movies.info())
print('\n------------\n')
print(ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
None

------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
None


In [7]:
print(movies.shape,'\n------------\n', ratings.shape)

(9742, 3) (100836, 4)


In [10]:
print(movies.describe(),'\n------------\n', ratings.describe())

             movieId
count    9742.000000
mean    42200.353623
std     52160.494854
min         1.000000
25%      3248.250000
50%      7300.000000
75%     76232.000000
max    193609.000000 
------------
               userId        movieId         rating     timestamp
count  100836.000000  100836.000000  100836.000000  1.008360e+05
mean      326.127564   19435.295718       3.501557  1.205946e+09
std       182.618491   35530.987199       1.042529  2.162610e+08
min         1.000000       1.000000       0.500000  8.281246e+08
25%       177.000000    1199.000000       3.000000  1.019124e+09
50%       325.000000    2991.000000       3.500000  1.186087e+09
75%       477.000000    8122.000000       4.000000  1.435994e+09
max       610.000000  193609.000000       5.000000  1.537799e+09


Из приведенной выше таблицы можно сделать вывод, что

    Средняя оценка составляет 3,5, а минимальная и максимальная - 0,5 и 5 соответственно.
    Всего 610 пользователей поставили свои оценки 193609 фильмам.